## CAMP KOREY | Capstone Project |  Last Modified: 9/1/2023

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import altair as alt
import datetime
import re

In [ ]:
alt.data_transformers.disable_max_rows()

### Naming convention for dataframes:


Donor biographical df = dnr_bio_df

Individual Donor biographical = InDnr_bio_df

Gifts dataset = gft_df

Individual gifts dataset = InDnr_Gft_df

Volunteer activity dataset = VA_df


## Data Cleaning - Donor Biographical data

In [ ]:
dnr_bio_df = pd.read_csv('Camp Korey Donor Biographical Data with Const ID.CSV',
                        encoding = 'unicode_escape',
                        low_memory = False)
dnr_bio_df.head()
# 22309 rows

* Deleting columns CnBio_Sort_name, CnCnstncy_1_01_CodeLong from dnr_bio_df() dataframe as they are duplicates

Note: 
* CnBio_Sort_name is same as CnBio_Name
* CnCnstncy_1_01_CodeLong is same as CnBio_Constit_Code


In [ ]:
dnr_bio_df = dnr_bio_df.drop(['CnBio_Sort_name','CnCnstncy_1_01_CodeLong'], axis=1)

In [ ]:
dnr_bio_df.info()

#### Transforming the Solicit code:

Went through and looked at unique codes for all solicit code fields 1-5. Then organized these codes by their meaning. 

*Not sure what Opt-In and Opt-Out mean at this point... Or Opt-In Cabin Chat, Opt-In Experience Camp Korey (are these newsletters?

In [ ]:
#looked at all unique values across the 5 solicit columns and created a full list
#used: dn_bio_df.CnSolCd_1_05_Solicit_Code.unique()
list_of_codes = ['Invalid Email', 'Do Not Solicit', 'Do Not Email',
       'No Contact At All', 'Do Not Mail', 'Do Not Contact',
       'Agreed to receive postal mail', 'Opt-In', 'Opt-Out',
       'Do Not Email, Do Not Call', 'DO NOT CALL. DO NOT CALL',
       'Opt-in Cabin Chat', 'Do Not Call', 'Send info on / invitations to events',
        'Opt-in Experience Camp Korey', 'Do not send info on events', 
        'Agreed to receive postal mail', 'Agreed to reach by phone']

#sorting by functional meaning:

dont_email = ['Do Not Solicit', 'No Contact At All', 'Do Not Contact', 'Invalid Email', 'Do Not Email', 
             'Do Not Email, Do Not Call', ]

invalid_email = ['Invalid Email']

dont_call = ['Do Not Solicit', 'No Contact At All', 'Do Not Contact', 'Do Not Email, Do Not Call', 
             'DO NOT CALL. DO NOT CALL', 'Do Not Call']

dont_mail = ['Do Not Solicit', 'No Contact At All', 'Do Not Contact', 'Do Not Mail', ]


Now I'm calculating three new fields which tell us if any of the 5 solicit columns contains an instruction that means they shouldn't be emailed/called/mailed. 
- dne = do not email
- dnc = do not call
- dnm = do not mail
- invalid_email

In [ ]:
#isin() function returns True if the row has a code in the "dont_email" list above
#then adding all of the boolean values:
#each row will have 'True' if they had a "dont email" code in ANY of the solicit columns
dnr_bio_df['dne'] = (
    np.isin(dnr_bio_df['CnSolCd_1_01_Solicit_Code'], dont_email)
    + np.isin(dnr_bio_df['CnSolCd_1_02_Solicit_Code'], dont_email)
    + np.isin(dnr_bio_df['CnSolCd_1_03_Solicit_Code'], dont_email)
    + np.isin(dnr_bio_df['CnSolCd_1_04_Solicit_Code'], dont_email)
    + np.isin(dnr_bio_df['CnSolCd_1_05_Solicit_Code'], dont_email)
) 

#same idea but for call and mail fields
dnr_bio_df['dnc'] = (
    np.isin(dnr_bio_df['CnSolCd_1_01_Solicit_Code'], dont_call)
    + np.isin(dnr_bio_df['CnSolCd_1_02_Solicit_Code'], dont_call)
    + np.isin(dnr_bio_df['CnSolCd_1_03_Solicit_Code'], dont_call)
    + np.isin(dnr_bio_df['CnSolCd_1_04_Solicit_Code'], dont_call)
    + np.isin(dnr_bio_df['CnSolCd_1_05_Solicit_Code'], dont_call)
) 

dnr_bio_df['dnm'] = (
    np.isin(dnr_bio_df['CnSolCd_1_01_Solicit_Code'], dont_mail)
    + np.isin(dnr_bio_df['CnSolCd_1_02_Solicit_Code'], dont_mail)
    + np.isin(dnr_bio_df['CnSolCd_1_03_Solicit_Code'], dont_mail)
    + np.isin(dnr_bio_df['CnSolCd_1_04_Solicit_Code'], dont_mail)
    + np.isin(dnr_bio_df['CnSolCd_1_05_Solicit_Code'], dont_mail)
) 

dnr_bio_df['invalid_email'] = (
    np.isin(dnr_bio_df['CnSolCd_1_01_Solicit_Code'], invalid_email)
    + np.isin(dnr_bio_df['CnSolCd_1_02_Solicit_Code'], invalid_email)
    + np.isin(dnr_bio_df['CnSolCd_1_03_Solicit_Code'], invalid_email)
    + np.isin(dnr_bio_df['CnSolCd_1_04_Solicit_Code'], invalid_email)
    + np.isin(dnr_bio_df['CnSolCd_1_05_Solicit_Code'], invalid_email)
) 

#### Normalizing the primary email id

In [ ]:
#Checking the list for Is_Primary columns and replacing Yes with True, and No with False

for is_primary_col in [ 'CnPh_1_01_Is_Primary', 'CnPh_1_02_Is_Primary','CnPh_1_03_Is_Primary', 'CnPh_1_04_Is_Primary','CnPh_1_05_Is_Primary', 'CnPh_1_06_Is_Primary']:
    
    dnr_bio_df[is_primary_col]= dnr_bio_df[is_primary_col].replace({'Yes': True, 'No': False})



In [ ]:
#creating a new primary email column

dnr_bio_df['prim_email'] = dnr_bio_df.apply(
    
    lambda row: next((row[phon_col] for phon_col in ['CnPh_1_01_Phone_type','CnPh_1_02_Phone_type','CnPh_1_03_Phone_type',  'CnPh_1_04_Phone_type', 'CnPh_1_05_Phone_type',  'CnPh_1_06_Phone_type'] 
                      
                      if phon_col.replace('Phone_type', 'Is_Primary') in row.index 
                      and row[ phon_col.replace('Phone_type', 'Is_Primary') ]), None),
    axis=1)
    

In [ ]:
#Drop all the phone_type columns and Is_Primary columns
dnr_bio_df =  dnr_bio_df.drop(['CnPh_1_01_Phone_type','CnPh_1_02_Phone_type','CnPh_1_03_Phone_type','CnPh_1_04_Phone_type','CnPh_1_05_Phone_type','CnPh_1_06_Phone_type','CnPh_1_01_Is_Primary','CnPh_1_02_Is_Primary','CnPh_1_03_Is_Primary','CnPh_1_04_Is_Primary','CnPh_1_05_Is_Primary','CnPh_1_06_Is_Primary'], axis=1)


In [ ]:
dnr_bio_df.head()

#1798 rows with primary email

#### Check for empty first and last names, with constituent code as "Individual," and update them to "Corporate."


In [ ]:
dnr_bio_df.CnBio_Constit_Code.unique()

In [ ]:
# Find the rows with empty first and last names and constituent code as "Individual"
condition_rows = ((dnr_bio_df['CnBio_First_Name'].isnull()) 
                 & (dnr_bio_df['CnBio_Last_Name'].isnull())  
                 & (dnr_bio_df['CnBio_Constit_Code'] == 'Individual'))

# Modify the constituent code for the matching rows to "Corporate"
dnr_bio_df.loc[condition_rows, 'CnBio_Constit_Code'] = 'Corporate'


In [ ]:
dnr_bio_df.head()

#### Generating a unique identifier value for all the blank fields in 'CnBio_ID' 

* Fetch the rows where CnBio_ID is empty and update those rows with a unique identifier value

In [ ]:
null_Bio_Id_records = dnr_bio_df[dnr_bio_df['CnBio_ID'].isnull()]
null_Bio_Id_records

* Drop the duplicate rows in the dataframe

In [ ]:
null_Bio_Id_records = null_Bio_Id_records.drop_duplicates(subset=['CnBio_Name'])

In [ ]:
null_Bio_Id_records

* Row with CnBio_Name as 'Traci Trocano' has CnBio_ID as '12/21/1959'. Therefore, update the CnBio_ID with a unique value

In [ ]:
#fetch the row where id is incorrect
trocano_modify = dnr_bio_df[dnr_bio_df['CnBio_Name']== 'Traci Trocano']

#Append the trocano_modify to the null_Bio_Id_records dataframe
null_Bio_Id_records = null_Bio_Id_records.append(trocano_modify, ignore_index = True)
null_Bio_Id_records


* Generate a unique identifier with 9 digits

In [ ]:
unq_ids = np.arange(10000, 10000+len(dnr_bio_df))

#set the seed in order to get same CnBio_ID even if we run multiple times
np.random.seed(37)
np.random.shuffle(unq_ids)

#appending unique id to 0000
unq_ids = ['0000' + str(unq_id) for unq_id in unq_ids]

#Updating all empty records with unique id
null_Bio_Id_records['CnBio_ID'] = unq_ids[:len(null_Bio_Id_records)]

In [ ]:
null_Bio_Id_records

In [ ]:
#saving the newly generated id's information
null_Bio_Id_records.to_csv('Records_With_NewIDs.csv',encoding='utf-8')

In [ ]:
dnr_bio_df

In [ ]:
# Update the dnr_bio_df with the values in null_Bio_Id_records
#dnr_bio_df.update(null_Bio_Id_records,overwrite=True)

In [ ]:
# Merge the CnBio_ID_x column based on the CnBio_ID_y column.

dn_merged_null = pd.merge(dnr_bio_df, null_Bio_Id_records[['CnBio_Name', 'CnBio_ID']], on='CnBio_Name', how='left')


In [ ]:
dn_merged_null.info()

In [ ]:
dn_merged_null['CnBio_ID_x'].update(dn_merged_null['CnBio_ID_y'])

In [ ]:
#check if the merging is done correctly for these donors
verify_users = dn_merged_null[dn_merged_null['CnBio_Name'].isin(['Lisa Eskenazy', 'Traci Trocano','David Boland'])]
verify_users

In [ ]:
# merge the CnBio_ID column with the actual dataset
dnr_bio_df['CnBio_ID'].update(dn_merged_null['CnBio_ID_x'])

In [ ]:
#verify these users again
verify_users = dnr_bio_df[dnr_bio_df['CnBio_Name'].isin(['Lisa Eskenazy', 'Traci Trocano',])]
verify_users

In [ ]:
dnr_bio_df

#### Saving the cleaned donor file

In [ ]:
dnr_bio_df.to_csv('Cleaned_Donor_bio.csv',encoding='utf-8')

*************************************************************

## Data Cleaning - Gifts Dataset

In [ ]:
#df_da : donor analysis
df_da = pd.read_csv('All Gifts for Donor Analysis.csv',encoding= 'unicode_escape')

* Filter to only 2020,2021,2022

In [ ]:
#Make a copy of donor analysis df first
gft_df = df_da.copy()

In [ ]:
# years we are interested in
yearlist = [2020,2021,2022]

In [ ]:
#Change Gf_Date to date datatype and extract year and month
gft_df['Gf_Date']= pd.to_datetime(df_da['Gf_Date'])
gft_df['year_donated'] = gft_df['Gf_Date'].dt.year
gft_df['month_donated'] = gft_df['Gf_Date'].dt.month

In [ ]:
gft_df.year_donated.unique()

In [ ]:
gft_df = gft_df[gft_df['year_donated'].isin(yearlist)]

In [ ]:
gft_df.year_donated.unique()

In [ ]:
gft_df.info()

* Filtering resulted in  6639 entries

* Clean up 'Gf_Amount' columns to hold numeric data only, 

In [ ]:
#Replace '$' symbol, replace ',' and change datatype to float for Gf_Amount
gft_df["Gf_Amount"]=gft_df["Gf_Amount"].str.replace("$","")
gft_df["Gf_Amount"]=gft_df["Gf_Amount"].str.replace(",","")

# change to datatype float
gft_df["Gf_Amount"] = gft_df["Gf_Amount"].astype(str).astype(float)
gft_df["Gf_Amount"]

* Delete 'Gf_Receipt_amount' and 'Gf_Gift_code' as requested by CK(2/14)

In [ ]:
#Delete Gf_Receipt_amount 
gft_df.drop('Gf_Receipt_amount', inplace=True, axis=1)
gft_df.drop('Gf_Gift_code', inplace=True, axis=1)

* Creating a new column Gf_Fund and renaming it to Gf_Fund_Old

In [ ]:
gft_df['Gf_Fund_Old'] = gft_df['Gf_Fund'].copy().rename('Gf_Fund_Old')


In [ ]:
gft_df.info()

#### Gf_Fund categories

In [ ]:
gft_df.Gf_Fund.unique()

In [ ]:
# Replacing all the categories that starts with 100 and EOY2020 to a new category "100: Fundraising"
gft_df.loc[gft_df['Gf_Fund'].str.startswith(('100','EOY2020')), 'Gf_Fund'] = '100:Fundraising'

gft_df.Gf_Fund.unique()

* Deleting columns Gf_CnAdrPrf_City2, Gf_CnAdrPrf_State2, Gf_CnAdrPrf_ZIP2 from df_gifts dataframe as they are duplicates

Note: 
* Gf_CnAdrPrf_City2 is the same as Gf_CnAdrPrf_City.
* Gf_CnAdrPrf_State2 is the same as Gf_CnAdrPrf_State. 
* Gf_CnAdrPrf_ZIP2 is the same as Gf_CnAdrPrf_ZIP. 

In [ ]:
gft_df = gft_df.drop(['Gf_CnAdrPrf_City2','Gf_CnAdrPrf_State2', 'Gf_CnAdrPrf_ZIP2'], axis=1)


#### Update the Gifts dataset with the unique identification number generated for empty_id_rows in Donor dataset.

* Update values of 'Gf_CnBio_ID' in Gifts dataset based on the generated unique id of Donor dataset

In [ ]:
for indexNo, rowdetails in null_Bio_Id_records.iterrows():
    donor_name = rowdetails['CnBio_Name']
    new_unique_id = rowdetails['CnBio_ID']
    gft_df.loc[gft_df['Gf_CnBio_Name'] == donor_name, 'Gf_CnBio_ID'] = new_unique_id


In [ ]:
#verify it for one donor
sam_moyda_row = gft_df[gft_df['Gf_CnBio_Name']== 'Sam Myoda']
sam_moyda_row

#### Saving the cleaned gifts file

In [ ]:
gft_df.to_csv('Cleaned_Gifts.csv',encoding='utf-8')

**********

## Exporting the missing donor data

#### Missing address information

In [ ]:
#Select the rows where there is no valid address
invalid_address = dnr_bio_df[dnr_bio_df['CnBio_No_Valid_Addresses'] == 'Yes']

#invalid_address.sort_values(by='col1', ascending=False)

#saving the missing donor address information
invalid_address.to_csv('missing_donor_address.csv',encoding='utf-8')

#Fetch top 5 rows
invalid_address.head(5)

#### Missing email information

In [ ]:
#Select the rows where there is an invalid email
invalid_email = dnr_bio_df[dnr_bio_df['invalid_email'] == True]

#saving the missing donor email information
invalid_email.to_csv('missing_donor_email.csv',encoding='utf-8')


# Fetch top 5 rows
invalid_email.head(5)

********

## Analysing Data

 * Sum of donations for each type of fund

In [ ]:
dnr_sum_by_fnd_typ = gft_df.groupby('Gf_Fund')['Gf_Amount'].sum()
dnr_sum_by_fnd_typ

* Sum of donations for each type of fund on yearly basis

In [ ]:
dnr_sum_by_fnd_typ_and_year = gft_df.groupby(['Gf_Fund','year_donated'])['Gf_Amount'].sum()
dnr_sum_by_fnd_typ_and_year

*****************

## Exploratory Analysis

In [ ]:
alt.Chart(gft_df).mark_bar().encode(
    alt.Y('count():Q'),
    alt.X('Gf_CnBio_Gender:N'),
    #alt.Color('year_donated:Q'),
    alt.Tooltip('count():Q')
).properties(
    width=400,
    height=250
)

In [ ]:
#df_da.Gf_Appeal.unique()

In [ ]:
bar = alt.Chart(gft_df).mark_bar().transform_filter(
    'datum.year_donated == 2021'
).encode(
    alt.X('count():Q'),
    alt.Y('Gf_Appeal:N',sort=alt.EncodingSortField(
        op='count', field='Gf_Appeal', order='descending')),
    #alt.Color('year_donated:Q'),
    alt.Tooltip('count():Q')
).properties(
    width=300,
    height=600,
    title = 'Gift Appeal count| Year 2021'
)

text = bar.mark_text(
    align='left',
    baseline='middle',
    dy=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='count():Q'
)

(bar + text).properties(
    width=300,
    height=600,
).configure_axis(
  labelAngle=0
)


In [ ]:
cumulative_amount_year = gft_df.groupby(['year_donated'])['Gf_Amount'].sum()
cumulative_amount_year

In [ ]:
alt.Chart(gft_df).mark_bar().encode(
    alt.X('year_donated:N', title='Year'),
    alt.Y('sum(Gf_Amount):Q', title='Total Donation Amount'),
    #alt.Color('year_donated:Q'),
    alt.Tooltip('sum(Gf_Amount):Q')
).properties(
    width=650,
    height=250,
    title=('Total amount donations by year')
).configure_axis(
  labelAngle=0
)

In [ ]:
alt.Chart(gft_df).mark_bar().encode(
    alt.Y('count():Q'),
    alt.X('Gf_Type:N'),
    alt.Tooltip('count():Q')
).properties(
    width=400,
    height=250
)

Most gifts are donated in 'Cash'. The varibles sound very similar. Have to filter by year